# Ground Truth Veri Seti Semantik Test Deneyleri

In [1]:
import json
import pandas as pd
from kufurnet import KufurNet

Toplamda 1.000 adet küfürlü ve küfürsüz metinler içeren etiketli "ground truth" json dosyasını yükleyelim.

In [2]:
with open("data/ground_truth.json", encoding="utf-8") as file:
    ground_truth = json.load(file)

**Bu veri seti twitter, facebook gibi sosyal medya platformlardan kazınmış olup içerdiği nefret söylemleri hakkında hiçbir sorumluluk kabul etmeyiz. Araştırma ve geliştirme süreçlerini desteklemesi amacıyla topladığımız bu veri setini çalışmalarınızda kullanarak sağlıklı değerlendirmeler elde etmeniz dileğiyle.**

In [3]:
ground_truth

{'ReadMe': 'Bu veri seti twitter, facebook gibi sosyal medya platformlardan kazınmış olup içerdiği nefret söylemleri hakkında hiçbir sorumluluk kabul etmeyiz. Araştırma ve geliştirme süreçlerini desteklemesi amacıyla topladığımız bu veri setini çalışmalarınızda kullanarak sağlıklı değerlendirmeler elde etmeniz dileğiyle.',
 '0': {'text': 'arkadaşlar eski sevim aşk acısı falan çekmiyomuş aq ben yanlış anlamışım mxcdmdmwkdxmfbm',
  'is_black': True,
  'black_words': ['aq']},
 '1': {'text': 'Kendisi çok yetenekli olmasına rağmen şehinşahın kendinden daha yetenekli olduğunu savunması hidra benim için o noktada bitti aga göt yalayan bir tip başka bişi deil',
  'is_black': True,
  'black_words': ['göt']},
 '2': {'text': '#nihatözdemiristifa ya paşa paşa gidersin yada biz seni göndermeyi biliriz sen git kara para akla onun bunun şirketinden mesela arkandaki ingiliz cocugunun yanına @TFF_Org',
  'is_black': False,
  'black_words': []},
 '3': {'text': "PKK'NIN PRAJESİ OLAN EŞBAŞKANLIK İBB'DE ÜS

Yorumlardan ne kadarı küfürlü ve ne kadarı temiz?

In [4]:
black_statuses = []

for i in range(1000):
    black_statuses.append(ground_truth[str(i)]["is_black"])

pd.Series(black_statuses).value_counts()

True     704
False    296
dtype: int64

Küfürlü yorumlar ezici çoğunlukta.

Ground Truth veri setinde test için Kufurnet sınıfını kullanacağız.

In [5]:
kufurnet = KufurNet()

Örnek bir küfürlü metin denemesi yapalım ve çıktının formatını görelim.

In [6]:
kufurnet.analyze({"text": "ulan ben senin yapacağın işi sikeyim ibnetor başgavat!"})

{'status': True,
 'result': {'is_black': True, 'black_score': 0.17, 'black_list': ['ulan']}}

Örnek bir küfürsüz metin denemesi yapalım ve çıktının formatını teyit edelim.

In [7]:
kufurnet.analyze({"text": "perşembe pilavlıda yoktun hayırdır mübarek?!"})

{'status': True,
 'result': {'is_black': False, 'black_score': 0.0, 'black_list': []}}

In [8]:
true_preds, false_preds = [], []

for i in range(1000): # Ground Truth verisinde 1000 adet text item olduğu için.
    gt_item = ground_truth[str(i)] # Her bir item'i dict olarak al.
    
    response = kufurnet.analyze({"text":gt_item["text"]}) # API'ye her bir item'in text'ini yolla.
    
    if response["result"]["is_black"] == gt_item["is_black"]: # API çıktısı ile Ground Truth eşleşiyor mu?
        true_preds.append(gt_item) # Eşleşiyorsa doğru tahmin.
    else:
        false_preds.append(gt_item) # Eşleşmiyorsa yanlış tahmin.

In [9]:
accuracy = (1 - (len(false_preds) / len(true_preds))) * 100
accuracy

98.06320081549438

In [10]:
false_preds

[{'text': 'Noluyo olum yunan yine mi kudurdu ne bu jetler?',
  'is_black': True,
  'black_words': ['kudurdu']},
 {'text': 'Ramazanda sırf bakıpta günaha girmeyelim diye memesiz kız paylaşan admin kral ya',
  'is_black': True,
  'black_words': ['memesiz']},
 {'text': '@user ateşleri bol olsun mayın eşekleri.',
  'is_black': True,
  'black_words': ['eşekleri']},
 {'text': '@ersoydede aynı fes değil biri yunanı denize döken sarı başbuğ daki fes,diğeri keşke yunan kazansaydı diyen fes. mesele fes değil içindeki zihniyet.ırzını namusunu çigeneyenlere sevicilik yapacak kadar alçakça giyilen fes ile trablusgarp,çanakkale de kurtuluş savaşındaki fes aynımı?',
  'is_black': True,
  'black_words': ['alçakça']},
 {'text': '@user aile malı nerdeyse çoğu en  azından açıklıyor sizin erdoğan da açıklasın da görelim sıkıyosa',
  'is_black': True,
  'black_words': ['sıkıyosa']},
 {'text': '"ekmeği dizinde" diye bir tabir var ya.. bizim milleti o iyi özetliyor. bu millete 1 öğün yemek 3 öğün sopa vuraca

In [11]:
for i in false_preds:
    print(i["black_words"])

['kudurdu']
['memesiz']
['eşekleri']
['alçakça']
['sıkıyosa']
[]
['sıktır']
['fetöcü']
['kodumun']
[]
['oropu']
['mal']
[]
['fetöcü']
[]
['bka']
[]
[]
[]
